In [1]:
#Import package
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib
from geopy.distance import geodesic
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from datetime import datetime, timedelta, date
# import pivottablejs as pvt
import math
import network_analysis_module

C:\Users\rmoumneh\AppData\Local\Temp\ipykernel_16984\2605363855.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# - Import data -
ambulances = pd.read_parquet('ambulance_locations.parquet.gzip')
postal_codes_subset = pd.read_csv('postal_codes.csv', sep=',')
postal_codes_subset.head()

,postcode,smun_code,reg_code,prov_code,arr_code,mun_code,Region,Province,Arrondissement,Municipality
0,3211,24066B,2000.0,20001.0,24000.0,24066.0,Flanders,Vlaams-Brabant,Leuven,Lubbeek
1,1910,23038A,2000.0,20001.0,23000.0,23038.0,Flanders,Vlaams-Brabant,Halle-Vilvoorde,Kampenhout
2,9120,46003A,2000.0,40000.0,46000.0,46003.0,Flanders,Oost-Vlaanderen,Sint-Niklaas,Beveren
3,9520,41063A,2000.0,40000.0,41000.0,41063.0,Flanders,Oost-Vlaanderen,Aalst,Sint-Lievens-Houtem
4,9667,45062B,2000.0,40000.0,45000.0,45062.0,Flanders,Oost-Vlaanderen,Oudenaarde,Horebeke


In [18]:
postal_codes_subset.columns
postal_codes_subset.dtypes

postcode            int64
smun_code          object
reg_code          float64
prov_code         float64
arr_code          float64
mun_code          float64
Region             object
Province           object
Arrondissement     object
Municipality       object
dtype: object

In [30]:
# ambulances
ambulances.head()
ambulances_cl=ambulances.copy()

In [31]:
regions = {'Brussels Hoofdstedelijk Gewest':'Brussels','Vlaams Gewest':'Flanders','Waals Gewest':'Wallonia','null':'null'}
ambulances_cl['region']=ambulances_cl['region'].map(regions)

for column in ambulances_cl.columns:
    if ambulances_cl[column].dtype=='str':
        ambulances_cl[column]=ambulances_cl[column].str.title().str.strip()
    else:
        ambulances_cl[column]=ambulances_cl[column]      
  
for index, row in ambulances_cl.iterrows():
    item = row['departure_location']
    strings = item.split()
    ambulances_cl.at[index,'departure_municipality']=item.split()[-1].title()
    for string in strings:
        if (len(string) == 4) and (string.isdigit()):
            ambulances_cl.at[index, 'departure_postalcode'] = string    



In [32]:
ambulances_cl.columns
ambulances_cl.dtypes

base                          object
medical_resource              object
province                      object
region                        object
departure_location            object
departure_location_number      int64
latitude                     float64
longitude                    float64
occasional_permanence         object
departure_municipality        object
departure_postalcode          object
dtype: object

In [34]:
# test 

ambulances_cl['departure_postalcode'].iloc[0]
ambulances_cl['departure_postalcode'] = ambulances_cl.apply(lambda x: int(x['departure_postalcode']), axis=1)

# End test 

In [35]:
ambulances_cl.columns
ambulances_cl.dtypes

base                          object
medical_resource              object
province                      object
region                        object
departure_location            object
departure_location_number      int64
latitude                     float64
longitude                    float64
occasional_permanence         object
departure_municipality        object
departure_postalcode           int64
dtype: object

In [36]:
ambulances_all = pd.merge(ambulances_cl,postal_codes_subset, left_on = 'departure_postalcode', right_on = 'postcode', how ='left')
for a, b in zip(ambulances_all.columns, range(0,len(ambulances_all.columns))):
                print(f'{a}, column number {b}')

ambulances_subset = ambulances_all.iloc[:,[0,2,3,4,5,6,7,9,10,17,18,19]]
ambulances_subset.head()

base, column number 0
medical_resource, column number 1
province, column number 2
region, column number 3
departure_location, column number 4
departure_location_number, column number 5
latitude, column number 6
longitude, column number 7
occasional_permanence, column number 8
departure_municipality, column number 9
departure_postalcode, column number 10
postcode, column number 11
smun_code, column number 12
reg_code, column number 13
prov_code, column number 14
arr_code, column number 15
mun_code, column number 16
Region, column number 17
Province, column number 18
Arrondissement, column number 19
Municipality, column number 20


,base,province,region,departure_location,departure_location_number,latitude,longitude,departure_municipality,departure_postalcode,Region,Province,Arrondissement
0,BA ANTW [Borgerhout] AMBUCE,Antwerpen,Flanders,Gijselstraat 150 2140 BORGERHOUT,3811,51.215516,4.443788,Borgerhout,2140,Flanders,Antwerpen,Antwerpen
1,BA ANTW [Hoboken] AMBUCE,Antwerpen,Flanders,Antwerpsesteenweg 242 2660 HOBOKEN,3812,51.178831,4.361948,Hoboken,2660,Flanders,Antwerpen,Antwerpen
2,BA ANTW [Hof Ter Schelde] AC Antwerpen,Antwerpen,Flanders,August Vermeylenlaan 6 2050 ANTWERPEN,10902,51.232810,4.377237,Antwerpen,2050,Flanders,Antwerpen,Antwerpen
3,BA ANTW [Kaai 606] SIHWA,Antwerpen,Flanders,Antwerpsebaan 4 (Kaai 606) 2040 ANTWERPEN,12902,51.315516,4.321506,Antwerpen,2040,Flanders,Antwerpen,Antwerpen
4,BA ANTW [Sint Elisabeth] AC Antwerpen,Antwerpen,Flanders,Leopoldstraat 26 2000 ANTWERPEN,10901,51.213623,4.405718,Antwerpen,2000,Flanders,Antwerpen,Antwerpen


In [37]:
ambulances_subset.to_csv('ambulances.csv', index=False)

In [39]:
ambulances_df = pd.read_csv('ambulances.csv', sep=',')
ambulances_df.head()

,base,province,region,departure_location,departure_location_number,latitude,longitude,departure_municipality,departure_postalcode,Region,Province,Arrondissement
0,BA ANTW [Borgerhout] AMBUCE,Antwerpen,Flanders,Gijselstraat 150 2140 BORGERHOUT,3811,51.215516,4.443788,Borgerhout,2140,Flanders,Antwerpen,Antwerpen
1,BA ANTW [Hoboken] AMBUCE,Antwerpen,Flanders,Antwerpsesteenweg 242 2660 HOBOKEN,3812,51.178831,4.361948,Hoboken,2660,Flanders,Antwerpen,Antwerpen
2,BA ANTW [Hof Ter Schelde] AC Antwerpen,Antwerpen,Flanders,August Vermeylenlaan 6 2050 ANTWERPEN,10902,51.232810,4.377237,Antwerpen,2050,Flanders,Antwerpen,Antwerpen
3,BA ANTW [Kaai 606] SIHWA,Antwerpen,Flanders,Antwerpsebaan 4 (Kaai 606) 2040 ANTWERPEN,12902,51.315516,4.321506,Antwerpen,2040,Flanders,Antwerpen,Antwerpen
4,BA ANTW [Sint Elisabeth] AC Antwerpen,Antwerpen,Flanders,Leopoldstraat 26 2000 ANTWERPEN,10901,51.213623,4.405718,Antwerpen,2000,Flanders,Antwerpen,Antwerpen


In [40]:
# Nearest Neighbour
# 1. Clean a bit more ambulance dataframe
ambulances_df.head(10)
amb_df = ambulances_df[['base','latitude','longitude','Arrondissement']]
# Create a Tuple 'coordonate' from latitude and longitude
amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
amb_df['coordonate'].head(10)
amb_df.head(10)

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

# 2. Create a list of coordonate (amb_list) 
# and a Mulitpoint of coordonate (departures)
# for Nearest Neighbour method
amb_list = amb_df['coordonate'].tolist() # TODO into function !!
departures= MultiPoint(amb_list)
# print(departures)

# Make to coordonate Tuple into Point
amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)
print(amb_df['PointCoordonate'].head(10))

# Test
testpoint = Point(50.5641805, 4.4907038)
amb_df.loc[amb_df['PointCoordonate'] == testpoint]
# print((testpoint.x,testpoint.y))

# # EXAMPLE NN method
# orig = Point(50.5,4.5)
# nearest_depart = nearest_points(orig, departures)
# near_idx0 = nearest_depart[0]
# near_idx1 = nearest_depart[1]
# print(nearest_depart)
# print(near_idx0)
# print(near_idx1)

# Adding new ambulance to the list of departures
# departures.append([Point(newCoordonate)])

0    POINT (51.2155158 4.4437875)
1    POINT (51.1788314 4.3619482)
2     POINT (51.2328096 4.377237)
3    POINT (51.3155156 4.3215063)
4    POINT (51.2136226 4.4057177)
5      POINT (51.319819 5.089912)
6    POINT (51.3209834 4.8566561)
7    POINT (51.1140555 4.6472462)
8    POINT (51.4646726 4.4725507)
9     POINT (51.0430641 4.815907)
Name: PointCoordonate, dtype: object


C:\Users\rmoumneh\AppData\Local\Temp\ipykernel_16984\3066386475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
C:\Users\rmoumneh\AppData\Local\Temp\ipykernel_16984\3066386475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)


,base,latitude,longitude,Arrondissement,coordonate,PointCoordonate
277,BF SART BHN AMBU,50.56418,4.490704,Nivelles,"(50.5641805, 4.4907038)",POINT (50.5641805 4.4907038)


In [ ]:
# Nearest Neighbour
# 1. Clean a bit more ambulance dataframe
ambulances_df.head(10)
amb_df = ambulances_df[['base','latitude','longitude','Arrondissement']]
# Create a Tuple 'coordonate' from latitude and longitude
amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
amb_df['coordonate'].head(10)
amb_df.head(10)

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

# 2. Create a list of coordonate (amb_list) 
# and a Mulitpoint of coordonate (departures)
# for Nearest Neighbour method
amb_list = amb_df['coordonate'].tolist() # TODO into function !!
departures= MultiPoint(amb_list)
# print(departures)

# Make to coordonate Tuple into Point
amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)
print(amb_df['PointCoordonate'].head(10))

# Test
testpoint = Point(50.5641805, 4.4907038)
amb_df.loc[amb_df['PointCoordonate'] == testpoint]
# print((testpoint.x,testpoint.y))

In [ ]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

def create_coorodnate_multipoint(data_columns):
    """
    Take a column of coordonate date (latitude,longitude) and return a Mulitpoint

    Input:
        data columns of coodonate (latitude,longitude)
    Output:
        Multipoint of coorodnates
    """
    data_list = data_columns.tolist()
    data_multipoint = MultiPoint(data_list)
    return data_multipoint

def coordonate_column(dataframe, data_latitude, data_longitude):
    """
    Add a 'coordonate' column in the dataframe based on the its latitude and longitude columns

    Input :
        column of latitude data (exemple: dataframe.latitude)
        column of longitude data (exemple: dataframe.longitude)
    
    Ouput :
        None
    """
    dataframe['coordonate'] = list(zip(data_latitude, data_longitude))